In [22]:
import os
import requests
import pymysql
from bs4 import BeautifulSoup

In [23]:
def mysql_connect(db_host, db_user, db_password, db_name, query, media_id, title, preview_txt, link, time, reporter):
    # MariaDB 연결 설정
    conn = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name,
    charset='utf8mb4'  # 한글 및 유니코드 문자 지원
    )
    # 커서 생성
    cursor = conn.cursor()
    #insert_query = "INSERT INTO news (media_id, title, preview, link, time, reporter) VALUES (%s, %s, %s, %s, %s, %s)"
    cursor.execute(query, (media_id, title, preview_txt, link, time, reporter))

    # 커밋으로 변경사항 저장
    conn.commit()

    # 연결 종료
    cursor.close()
    conn.close()

In [19]:
def crawling_news():
    # url = 'https://www.newsis.com/realnews/?cid=realnews&day=today&page=1'
    # response = requests.get(url)
    base_url = 'https://www.newsis.com/realnews/?cid=realnews&day=today&page='
    first_page_url = f'{base_url}1'
    response = requests.get(first_page_url)

    if response.status_code != 200:
        print("Failed to get data")
        return

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')
    with open('output_file.txt', 'w', encoding='utf-8') as f:
        f.write(str(soup))
        
    # 'id'가 'paging_t1'인 div 태그의 내용을 가져오기
    last_button_link_direct = soup.find('div', id='paging_t1').find('button', class_='end').find_parent('a')
    last_page_number = last_button_link_direct['href'].split('page=')[-1] if last_button_link_direct else None
    # print(last_page_number)

    db_host = os.getenv('DB_HOST')
    db_user = os.getenv('DB_USER')
    db_password = os.getenv('DB_PASSWORD')
    db_name = os.getenv('DB_NAME')
    media_id = 1
    
    # 모든 페이지를 순회하며 뉴스 정보 크롤링
    for page in range(1, int(last_page_number) + 1):
        page_url = f'{base_url}{page}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # '.txtCont' 셀렉터를 사용하여 각 뉴스 블록 선택
        news_blocks = soup.select('.txtCont')

        for block in news_blocks:
            title_link = block.select_one('.tit a')
            preview = block.select_one('.txt a')
            if title_link and preview:
                preview_txt = preview.text.strip()
                title = title_link.text.strip()
                link = title_link['href']
                # 시간 정보와 기자 이름 추출 로직 개선
                reporter = block.select_one('.time').span.text.strip()
                time = block.select_one('.time')
                if time.span:
                    time.span.decompose()
                time = time.text.strip()
                print(f'Title: {title}\nPreview: {preview_txt}\nLink: {link}\nTime: {time}\nReporter: {reporter}\n')
                query = "INSERT INTO news (media_id, title, preview, link, time, reporter) VALUES (%s, %s, %s, %s, %s, %s)" 
                mysql_connect(db_host, db_user, db_password, db_name, query, media_id, title, preview_txt, link, time, reporter)
                print("-----------------------------------------------------------------------")


In [20]:
crawling_news()

Title: 한동훈 "이재명, 김준혁의 음담패설 옹호하는 글 올려"
Preview: 한동훈 국민의힘 비상대책위원장은 8일 "김준혁 후보의 말도 안되는 음담패설에 대해 오늘 이재명 더불어민주당 대표가 SNS에 그걸 옹호하는 취지의 글을 올렸다"고 비판했다.

한 위원장은 이날 저녁 경기 김포 유세현장에서 "김준혁 후보와 양문석 후보, 굉장히 오랫동안 대한민국 사람들이 강력히 비판, 비난했는데 정말 놀랍다&quo
Link: /view/?id=NISX20240408_0002692718
Time: 2024.04.08 20:44:11
Reporter: 신항섭기자

-----------------------------------------------------------------------
Title: 이재명, 문재인 혐오 발언한 국힘 윤영석에 "즉각 물러나야"
Preview: 이재명 더불어민주당 대표는 8일 선거 유세 중 문재인 전 대통령을 향해 막말을 한 윤영석 국민의힘 경남 양산갑 후보의 사퇴를 촉구했다.

이 대표는 이날 오후 자신의 사회관계망서비스(SNS) 페이스북에 "믿기 힘든 극언에 등골이 서늘했다"며 "전직 대통령을 상대로 폭력과 테러를 부추기는 집권여당 후보라니, 대체 민주주의를
Link: /view/?id=NISX20240408_0002692717
Time: 2024.04.08 20:43:35
Reporter: 강주희기자

-----------------------------------------------------------------------
Title: 전국법관대표회의, 사법행정자문위 운영 검토(종합)
Preview: 전국 법관들이 모여 사법행정자문회의 대안으로 법원조직법에 근거를 둔 사법정책자문위원회 운영을 검토하기로 했다.

전국법관대표회의는 8일 오전 10시부터 경기도 고양시 일산동구의 사법연수원에서 '전국법관대표회의 2024년 상반기 정기회의'를 통해 이 같은 안건을 논의했다고 밝혔다.

전국법관대표회의는 각급 법원에서 선출된 대표 판